In [404]:
import os
import glob
import pandas as pd
import numpy as np
import datetime

In [405]:
#s

## Setup list of columns for summary DF

In [430]:
columns_list=['Month','Total Trips','Total Stations','Trips per Station','Highest Start Station', 'Highest End Station','Total Bikes Used','Bike per Station','Trips per Bike','Female Ratio','Unknown Gender Ratio','Mode Age', 'Mode Age Ratio', 'Subscriber Ratio', 'Mean Trip Duration', 'Median Trip Duration']


## Setup Summary DF 

In [451]:
city_summary=pd.DataFrame(columns=columns_list)
city_summary
list_columns=['tripduration',
 'starttime',
 'stoptime',
 'start station id',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender']

# Read files

In [452]:
path ='D:/Yakov/099_DATA_Science/citybike'
for filename in glob.glob(os.path.join(path, '*.csv')):
    print(filename)
    #read file to csv
    df=pd.read_csv(filename)
    #cleanup \N from age
    df.replace(r'\\N','none',regex=True,inplace=True)
    df.columns=list_columns
    #total trips
    total_trips=len(df)
    #month year
    try_date=df['starttime'][0].split(" ",1)[0]
    try:
        date_format='%m/%d/%Y'
        date_time_obj=datetime.datetime.strptime(try_date,date_format)
    except:
        date_format='%Y-%m-%d'
        date_time_obj=datetime.datetime.strptime(try_date,date_format)
    month=date_time_obj.date()
    year=date_time_obj.date().year
    #stations
    start_station_stats=df['start station name'].value_counts()
    total_stations=len(start_station_stats)
    trips_station=round(total_trips/total_stations,1)
    start_station=start_station_stats.index[0]
    #end station
    end_station_stats=df['end station name'].value_counts()
    end_station=end_station_stats.index[0]
    #bikes
    bikes_stats=df['bikeid'].value_counts()
    bikes_describe=bikes_stats.describe()
    total_bikes=len(bikes_stats)
    bike_station=round(total_bikes/total_stations,1)
    trips_bikes=round(bikes_describe['mean'],1)
    median_bike_trips=round(bikes_describe['50%'])
    #gender
    gender_stats=df['gender'].value_counts()
    female=round(gender_stats[2]/total_trips,3)
    unknown=round(gender_stats[0]/total_trips,3)
    #age
    #1969 fix - average

    age_stats=df['birth year'].value_counts()
    age=df.groupby(['birth year']).agg({'bikeid':'count'})
    try :
        age.loc[1969]=(age.loc[1966:1968].mean()+age.loc[1970:1972].mean())/2
        age.reset_index(inplace=True)
    except:
        print('exception')
        age.reset_index(inplace=True)
        age.drop(age.loc[age['birth year']=='none'].index,inplace=True)
    #age binning
    age['age']=year-age['birth year'].astype(int)
    labels=['<20','20-29','30-39','40-49','50-59','60-69','70+']
    bins=[0,20,30,40,50,60,70,500]
    age['age_group']=pd.cut(age['age'], bins=bins, labels=labels)
    age_agg=age.groupby('age_group',as_index=False).agg({'bikeid':'sum'}).sort_values('bikeid',ascending=False,ignore_index=True)
    mode_age=age_agg.age_group[0]
    mode_age_ratio=round(age_agg.bikeid[0]/age_agg.bikeid.sum(),3)
    #user type
    usertype_stats=df['usertype'].value_counts()
    subscriber=round(usertype_stats.Subscriber/sum(usertype_stats),3)
    #trip duration
    durations_stats=df['tripduration'].describe()
    duration_mean=round(durations_stats['mean']/60,2)
    duration_median=round(durations_stats['50%']/60,2)
    # row creation
    new_row=[month,total_trips,total_stations,trips_station,start_station,end_station,total_bikes,bike_station,trips_bikes,female,unknown,mode_age,mode_age_ratio,subscriber,duration_mean,duration_median]
    # append Df
    city_summary.loc[len(city_summary)]=new_row
    
        

D:/Yakov/099_DATA_Science/citybike\2013-07 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2013-08 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2013-09 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2013-10 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2013-11 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2013-12 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-01 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-02 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-03 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-04 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-05 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-06 - Citi Bike trip data.csv
exception
D:/Yakov/099_DATA_Science/citybike\2014-07 - Citi Bi

In [453]:
city_summary.to_csv('city_summary_part1.csv',index=False)

In [454]:
city_summary

,Month,Total Trips,Total Stations,Trips per Station,Highest Start Station,Highest End Station,Total Bikes Used,Bike per Station,Trips per Bike,Female Ratio,Unknown Gender Ratio,Mode Age,Mode Age Ratio,Subscriber Ratio,Mean Trip Duration,Median Trip Duration
0,2013-07-01,843416,329,2563.6,W 20 St & 11 Ave,E 17 St & Broadway,5461,16.6,154.4,0.186,0.207,30-39,0.351,0.793,16.64,12.15
1,2013-08-01,1001958,330,3036.2,E 17 St & Broadway,E 17 St & Broadway,5750,17.4,174.3,0.199,0.186,30-39,0.348,0.814,16.33,12.07
2,2013-09-01,1034359,330,3134.4,W 20 St & 11 Ave,E 17 St & Broadway,5444,16.5,190.0,0.212,0.140,30-39,0.327,0.860,15.08,11.22
3,2013-10-01,1037712,330,3144.6,8 Ave & W 31 St,E 17 St & Broadway,5722,17.3,181.4,0.218,0.094,20-29,0.316,0.906,13.76,10.37
4,2013-11-01,675774,330,2047.8,8 Ave & W 31 St,E 17 St & Broadway,5978,18.1,113.0,0.214,0.065,20-29,0.313,0.935,12.85,9.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2020-11-01,1736704,1152,1507.6,1 Ave & E 68 St,1 Ave & E 68 St,18993,16.5,91.4,0.286,0.105,20-29,0.335,0.784,19.18,11.77
89,2020-12-01,1088929,1174,927.5,1 Ave & E 68 St,1 Ave & E 68 St,16841,14.3,64.7,0.267,0.087,30-39,0.296,0.845,16.87,10.27
90,2021-01-01,1095346,1221,897.1,1 Ave & E 68 St,1 Ave & E 68 St,14720,12.1,74.4,0.265,0.083,30-39,0.317,0.863,15.57,9.67
91,2021-02-01,634631,1274,498.1,1 Ave & E 68 St,1 Ave & E 68 St,15770,12.4,40.2,0.073,0.730,30-39,0.293,0.262,17.40,9.88
